In [ ]:
%pip install -r requirements.txt -q

In [ ]:
from phoenix.otel import register

tracer_provider = register(
  project_name="default",
  endpoint="http://127.0.0.1:6006/v1/traces"
)

#python -m phoenix.server.main serve

In [ ]:
import os
from smolagents import CodeAgent
from smolagents import OpenAIServerModel

from dotenv import load_dotenv
load_dotenv()

In [ ]:
from phoenix.otel import register
from openinference.instrumentation.smolagents import SmolagentsInstrumentor

register()
SmolagentsInstrumentor().instrument()

In [ ]:
def load_prompt_from_file(filepath):
    """Загружает текст промпта из указанного файла."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"ОШИБКА: Файл промпта не найден: {filepath}")
        return None
    except Exception as e:
        print(f"ОШИБКА: Не удалось прочитать файл промпта {filepath}: {e}")
        return None

In [ ]:
model = OpenAIServerModel(
    model_id="gpt-4.1-mini",
    api_base="https://api.openai.com/v1/",
    api_key=os.environ["OPENAI_API_KEY"],
)

In [ ]:
ml_specialist_agent = CodeAgent(
    tools=[],
    model=model,
    additional_authorized_imports=['numpy', 'pandas', 'sklearn', 'catboost'], 
    name="ml_specialist_agent",
    description="Specializes in training models, making predictions, and evaluating performance using ROC-AUC metric from sklearn.metrics. Cannot use numpy.random, might use internal model seeds if possible."
)
ml_specialist_agent.prompt_templates["system_prompt"] = load_prompt_from_file("ml_system_prompt.txt")

In [ ]:
project_manager_agent = CodeAgent(
    tools=[],
    model=model,
    additional_authorized_imports=['numpy', 'pandas', 'sklearn.metrics, seaborn, sklearn.model_selection, sklearn.preprocessing, sklearn.linear_model'],
    name="project_manager_agent",
    description="Manages data science projects for churn prediction. Breaks down tasks, delegates preprocessing and model training/evaluation to ml_specialist_agent, coordinates their work, synthesizes results, and provides the final ROC-AUC score.",
    managed_agents=[ml_specialist_agent]
)


project_manager_agent.prompt_templates["system_prompt"] = load_prompt_from_file("pm_system_prompt.txt")

In [ ]:

# task = '''You've been given the files:  
# - user_churn/train.csv: training data  
# - user_churn/test.csv: data for testing  

# Train a Logistic Regression model to predict customer churn ('Churn' column).    
# Test the model on the test dataset and report the ROC-AUC score.'''  

task = '''Your task is to train a machine learning model and measure the metric on a test set (regression task).
You are given 'flat_price/train_flat.csv' (training data) and 'flat_price/test_flat.csv' (testing data).

You can train any models from libraries and frameworks that are given to you.
You need to predict the target variable 'price'.
After training, evaluate the quality on a test set.
Evaluate the trained model on the test dataset using the RMSE metric.
As a final answer, give: 
1. The value of the metric 
2. A description of how the data was processed and the model was trained.
'''



result = project_manager_agent.run(
    task,
)

In [ ]:
result